# Priprava podatkov

In [30]:
import pandas as pd
df = pd.read_spss("Baza.sav")

In [31]:
df.head()

,šifra,spol,starost_leta,starost_meseci,ustanova,ocenjevalec,psiholog,datum,t1,t2,...,Ž_zapisovanje,D_zapisovanje,Š_zapisovanje,E_zapisovanje,H_zapisovanje,Č_zapisovanje,Kazanje,Prepoznavanje,Ocena,drugoo
0,01A08,moški,6.0,6.0,OŠ OB RINŽI KOČEVJE,"MATEJA VIDRIH, MAJA ŽAGAR",,2023-05-17,pogosto,pogosto,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1,01B04,ženski,7.0,1.0,OŠ B RINŽI KOČEVJE,"TANJA DOVŽAN, MOJCA BOC TURK",,2023-05-24,6.0,nikoli,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2,A1A01,moški,6.0,9.0,OŠ ANTONA UKMARJA,VALENTINA BOLČIČ,,2023-05-30,6.0,nikoli,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,
3,A1A02,moški,7.0,1.0,OŠ ANTONA UKMARJA,VALENTINA BOLČIČ,,2023-05-30,5.0,včasih,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,
4,A1A03,moški,6.0,8.0,OŠ ANTONA UKMARJA,VALENTINA BOLČIČ,,2023-05-30,vedno,pogosto,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,


In [32]:
num_rows = df.shape[0]

print("Number of rows in DataFrame:", num_rows)

Number of rows in DataFrame: 826


Stolpce, ki manjka več kot petina podatkov izbrišemo

In [33]:
manjkajoce = df.isnull().sum()

meja = 0.3
counter = 0 #steje koliko stolpcev smo pometali ven
for col in df.columns:
    if manjkajoce[col] > 0:
        if manjkajoce[col] < len(df)*meja:
            pass
        else:
            print(col, round(manjkajoce[col]/len(df[col]),2))
            df.drop(col, axis=1, inplace=True)
            counter += 1

status_matere2 0.32
status_očeta2 0.34


In [34]:
meja_vrstice = 0.3 #manjka vec kot 70% v vrstici
vse_vrstice = df.shape[0]
df = df[df.isnull().sum(axis=1) < meja_vrstice * len(df.columns)]
nove_vrstice = df.shape[0]
print("Pometane vrstice (v procentih)", round(((vse_vrstice - nove_vrstice) / vse_vrstice),2))

#Koliko stolpcev smo pometali ven
print("Pometani stolpci (v procentih)", round(((counter + 1) / (len(df.columns)+counter)),2))

Pometane vrstice (v procentih) 0.17
Pometani stolpci (v procentih) 0.01


In [35]:
# Izračunamo starost v mesecih
df['starost_leta'] = pd.to_numeric(df['starost_leta'], errors='coerce')
df['starost_meseci'] = pd.to_numeric(df['starost_meseci'], errors='coerce')

# Fill NaN values in numeric columns with mean
numeric_columns = ['starost_leta', 'starost_meseci']
df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].mean())

# Round 1.5 to 2
df['starost_meseci'] = df['starost_meseci'].apply(lambda x: 2 if x == 1.5 else x)

# Calculate age in months
df['age_in_months'] = df['starost_leta'] * 12 + df['starost_meseci']

df.drop(columns=['starost_leta', 'starost_meseci'], inplace=True)


In [36]:
column_names = df.columns.tolist()

# Print column names
print(column_names)

unique_values = df['jezik'].unique()

print("Unique values in 'Column1':", unique_values)

['šifra', 'spol', 'ustanova', 'ocenjevalec', 'psiholog', 'datum', 't1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12', 't13', 't14', 't15', 't16', 't17', 't18', 't19', 't20', 't21', 't22', 't23', 't24', 't25', 't26', 't27', 't28', 't29', 't30', 't31', 't32', 't33', 't34', 't35', 't36', 't37', 't38', 't39', 't40', 't41', 't42', 't43', 't44', 't45', 't46', 't47', 't48', 't49', 't50', 't51', 't52', 't53', 't54', 't55', 't56', 't57', 't58', 't59', 't60', 't61', 't62', 't63', 't64', 't65', 't66', 't67', 't68', 't69', 't70', 't71', 't72', 't73', 't74', 't75', 't76', 't77', 't78', 't79', 't80', 'spol_starša', 'izob_matere', 'izob_očeta', 'število', 'izpolnjeval', 'vključen', 'leta', 'meseci', 'status_matere', 'status_očeta', 'jezik', 'OPP', 'v10_1', 'v10_2', 'v10_3', 'v10_4', 'v10_5', 'v10_6', 'v10_7', 'v10_8', 'v10_9', 'v10_10', 'v10_11', 'v10_12', 'v10_13', 'v10_14', 'v11_1', 'v11_2', 'v11_3', 'v11_4', 'v11_5', 'v11_6', 'v11_7', 'v11_8', 'v12_1', 'v12_2', 'v12_3', 'v12

In [37]:
valid_values = [0, 1, 2, 3, 4,5,6]#Nekateri so leta pisali cudno...

# Remove values in 'leta' column that are not valid
df = df[df['leta'].isin(valid_values)]

if 'vključen' in df.columns and 'leta' in df.columns and 'meseci' in df.columns:
    df['leta'] = pd.to_numeric(df['leta'], errors='coerce')
    df['meseci'] = pd.to_numeric(df['meseci'], errors='coerce')
    
    # Fill NaN values in numeric columns with mean
    numeric_columns = ['leta', 'meseci']
    df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].mean())


    # Calculate age in months
    df['starost_vkljucitev'] = df['leta'] * 12 + df['meseci']

    # Drop 'leta' and 'meseci' columns
    df.drop(columns=['leta', 'meseci'], inplace=True)

In [38]:
nan_count = df['vključen'].isna().sum()
print("Number of NaN values in 'vključen' column:", nan_count)

# Drop rows with NaN values in 'vključen' column
df.dropna(subset=['vključen'], inplace=True)

Number of NaN values in 'vključen' column: 3


In [39]:
likert_mapping = {
    'niti se ne strinjam, niti se strinjam': 0,
    'popolnoma se strinjam': 2,
    'se ne strinjam': -1,
    'sploh se ne strinjam': -2,
    'strinjam se': 1
}

In [40]:
import pingouin as pg

v10_cols = ['v10_1', 'v10_2', 'v10_3', 'v10_4', 'v10_5',
            'v10_6', 'v10_7', 'v10_8', 'v10_9', 'v10_10',
            'v10_11', 'v10_12', 'v10_13', 'v10_14']

df[v10_cols] = df[v10_cols].replace(likert_mapping)

df[v10_cols] = df[v10_cols].apply(pd.to_numeric, errors='coerce')
df[v10_cols] = df[v10_cols].fillna(df[v10_cols].mean())

# (2) Calculate Cronbach's alpha using pingouin
alpha = pg.cronbach_alpha(df[v10_cols])
print("Cronbach's alpha:", alpha)


df['V10'] = df[v10_cols].mean(axis=1)

df.drop(columns=v10_cols, inplace=True)

Cronbach's alpha: (0.9510501952919679, array([0.945, 0.956]))


In [41]:
v11_cols = ['v11_1', 'v11_2', 'v11_3', 'v11_4', 'v11_5', 'v11_6', 'v11_7', 'v11_8']

df[v11_cols] = df[v11_cols].replace(likert_mapping)

df[v11_cols] = df[v11_cols].apply(pd.to_numeric, errors='coerce')
df[v11_cols] = df[v11_cols].fillna(df[v11_cols].mean())

# (2) Calculate Cronbach's alpha using pingouin
alpha = pg.cronbach_alpha(df[v11_cols])
print("Cronbach's alpha:", alpha)


df['V11'] = df[v11_cols].mean(axis=1)

df.drop(columns=v11_cols, inplace=True)

Cronbach's alpha: (0.8900710672991373, array([0.877, 0.902]))


In [42]:
v12_cols = ['v12_1', 'v12_2', 'v12_3', 'v12_4', 'v12_5', 'v12_6', 'v12_7']

df[v12_cols] = df[v12_cols].replace(likert_mapping)

df[v12_cols] = df[v12_cols].apply(pd.to_numeric, errors='coerce')
df[v12_cols] = df[v12_cols].fillna(df[v12_cols].mean())

# (2) Calculate Cronbach's alpha using pingouin
alpha = pg.cronbach_alpha(df[v12_cols])
print("Cronbach's alpha:", alpha)


df['V12'] = df[v12_cols].mean(axis=1)

df.drop(columns=v12_cols, inplace=True)

Cronbach's alpha: (0.8678710551926316, array([0.852, 0.883]))


In [43]:
v13_cols = ['v13_1', 'v13_2', 'v13_3', 'v13_4', 'v13_5']

df[v13_cols] = df[v13_cols].replace(likert_mapping)

df[v13_cols] = df[v13_cols].apply(pd.to_numeric, errors='coerce')
df[v13_cols] = df[v13_cols].fillna(df[v13_cols].mean())

# (2) Calculate Cronbach's alpha using pingouin
alpha = pg.cronbach_alpha(df[v13_cols])
print("Cronbach's alpha:", alpha)


df['V13'] = df[v13_cols].mean(axis=1)

df.drop(columns=v13_cols, inplace=True)

Cronbach's alpha: (0.8703688446834695, array([0.854, 0.885]))


In [44]:
v14_cols = ['v14_1', 'V14_2', 'v14_3', 'v14_4', 'v14_5', 'v14_6', 'v14_7', 'v14_8']

df[v14_cols] = df[v14_cols].replace(likert_mapping)

df[v14_cols] = df[v14_cols].apply(pd.to_numeric, errors='coerce')
df[v14_cols] = df[v14_cols].fillna(df[v14_cols].mean())

# (2) Calculate Cronbach's alpha using pingouin
alpha = pg.cronbach_alpha(df[v14_cols])
print("Cronbach's alpha:", alpha)


df['V14'] = df[v14_cols].mean(axis=1)

df.drop(columns=v14_cols, inplace=True)

Cronbach's alpha: (0.7946905100044857, array([0.77 , 0.818]))


In [45]:
v15_cols = ['v15_1', 'v15_2', 'v15_3', 'v15_4']

df[v15_cols] = df[v15_cols].replace(likert_mapping)

df[v15_cols] = df[v15_cols].apply(pd.to_numeric, errors='coerce')
df[v15_cols] = df[v15_cols].fillna(df[v15_cols].mean())

# (2) Calculate Cronbach's alpha using pingouin
alpha = pg.cronbach_alpha(df[v15_cols])
print("Cronbach's alpha:", alpha)


df['V15'] = df[v15_cols].mean(axis=1)

df.drop(columns=v15_cols, inplace=True)

Cronbach's alpha: (0.4973369317633365, array([0.432, 0.557]))


In [46]:
v16_cols = ['v16_1', 'v16_2', 'v16_3', 'v16_4']

df[v16_cols] = df[v16_cols].replace(likert_mapping)

df[v16_cols] = df[v16_cols].apply(pd.to_numeric, errors='coerce')
df[v16_cols] = df[v16_cols].fillna(df[v16_cols].mean())

# (2) Calculate Cronbach's alpha using pingouin
alpha = pg.cronbach_alpha(df[v16_cols])
print("Cronbach's alpha:", alpha)


df['V16'] = df[v16_cols].mean(axis=1)

df.drop(columns=v16_cols, inplace=True)

Cronbach's alpha: (0.5747176359693507, array([0.519, 0.625]))


In [47]:
v17_cols = ['v17_1', 'v17_2', 'v17_3', 'v17_4', 'v17_5', 'v17_6']

df[v17_cols] = df[v17_cols].replace(likert_mapping)

df[v17_cols] = df[v17_cols].apply(pd.to_numeric, errors='coerce')
df[v17_cols] = df[v17_cols].fillna(df[v17_cols].mean())

# (2) Calculate Cronbach's alpha using pingouin
alpha = pg.cronbach_alpha(df[v17_cols])
print("Cronbach's alpha:", alpha)


df['V17'] = df[v17_cols].mean(axis=1)

df.drop(columns=v17_cols, inplace=True)

Cronbach's alpha: (0.7191198205380439, array([0.685, 0.751]))


In [48]:
v18_cols = ['v18_1', 'v18_2', 'v18_3', 'v18_4', 'v18_5', 'v18_6', 'v18_7']

df[v18_cols] = df[v18_cols].replace(likert_mapping)

df[v18_cols] = df[v18_cols].apply(pd.to_numeric, errors='coerce')
df[v18_cols] = df[v18_cols].fillna(df[v18_cols].mean())

# (2) Calculate Cronbach's alpha using pingouin
alpha = pg.cronbach_alpha(df[v18_cols])
print("Cronbach's alpha:", alpha)


df['V18'] = df[v18_cols].mean(axis=1)

df.drop(columns=v18_cols, inplace=True)

Cronbach's alpha: (0.6586782894141963, array([0.617, 0.697]))


In [49]:
v19_cols = ['v19_1', 'v19_2', 'v19_3', 'v19_4', 'v19_5', 'v19_6', 'v19_7', 'v19_8', 'v19_9', 'v19_10', 'v19_11', 'v19_12']

df[v19_cols] = df[v19_cols].replace(likert_mapping)

df[v19_cols] = df[v19_cols].apply(pd.to_numeric, errors='coerce')
df[v19_cols] = df[v19_cols].fillna(df[v19_cols].mean())

# (2) Calculate Cronbach's alpha using pingouin
alpha = pg.cronbach_alpha(df[v19_cols])
print("Cronbach's alpha:", alpha)


df['V19'] = df[v19_cols].mean(axis=1)

df.drop(columns=v19_cols, inplace=True)

Cronbach's alpha: (0.8913629816410432, array([0.879, 0.903]))


In [50]:
df.head()

,šifra,spol,ustanova,ocenjevalec,psiholog,datum,t1,t2,t3,t4,...,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19
3,A1A02,moški,OŠ ANTONA UKMARJA,VALENTINA BOLČIČ,,2023-05-30,5.0,včasih,6.0,nikoli,...,2.000000,2.000,2.000000,2.0,2.000,2.00,0.50,1.500000,2.0,2.000000
4,A1A03,moški,OŠ ANTONA UKMARJA,VALENTINA BOLČIČ,,2023-05-30,vedno,pogosto,5.0,nikoli,...,1.000000,1.000,1.000000,1.0,1.000,1.00,1.00,1.000000,1.0,1.000000
5,A1A04,moški,OŠ ANTONA UKMARJA,VALENTINA BOLČIČ,,2023-05-30,6.0,5.0,6.0,nikoli,...,2.000000,1.750,1.857143,2.0,1.875,2.00,1.25,1.666667,2.0,1.833333
6,A1A05,moški,OŠ ANTONA UKMARJA,VALENTINA BOLČIČ,,2023-05-30,6.0,nikoli,6.0,nikoli,...,1.142857,0.250,0.142857,0.4,1.000,1.00,0.50,1.000000,2.0,1.583333
7,A1A06,ženski,OŠ ANTONA UKMARJA,VALENTINA BOLČIČ,,2023-05-30,5.0,včasih,6.0,nikoli,...,1.928571,1.125,1.428571,1.0,1.625,1.75,1.00,0.166667,1.0,0.833333


In [51]:
num_rows = df.shape[0]

print("Number of rows in DataFrame:", num_rows)

Number of rows in DataFrame: 658


In [52]:
column_names = df.columns.tolist()

# Print column names
print(column_names)

unique_values = df['jezik'].unique()

print("Unique values in 'Column1':", unique_values)

['šifra', 'spol', 'ustanova', 'ocenjevalec', 'psiholog', 'datum', 't1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12', 't13', 't14', 't15', 't16', 't17', 't18', 't19', 't20', 't21', 't22', 't23', 't24', 't25', 't26', 't27', 't28', 't29', 't30', 't31', 't32', 't33', 't34', 't35', 't36', 't37', 't38', 't39', 't40', 't41', 't42', 't43', 't44', 't45', 't46', 't47', 't48', 't49', 't50', 't51', 't52', 't53', 't54', 't55', 't56', 't57', 't58', 't59', 't60', 't61', 't62', 't63', 't64', 't65', 't66', 't67', 't68', 't69', 't70', 't71', 't72', 't73', 't74', 't75', 't76', 't77', 't78', 't79', 't80', 'spol_starša', 'izob_matere', 'izob_očeta', 'število', 'izpolnjeval', 'vključen', 'status_matere', 'status_očeta', 'jezik', 'OPP', 'drugo', 'P1', 'opažanja_P1', 'P2', 'opažanja_P2', 'P3', 'opažanja_P3', 'P4', 'opažanja_P4', 'P5', 'opažanja_P5', 'P6', 'opažanja_P6', 'R', 'M', 'P', 'G', 'F', 'A', 'Z', 'S', 'T', 'N', 'B', 'I', 'O', 'K', 'C', 'U', 'L', 'V', 'J', 'Ž', 'D', 'Š', 'E', 'H

In [53]:
# Define mapping of values to be replaced
replace_dict = {
    'Slovenski jezik': 'Slovenščina',
    'Slovenski': 'Slovenščina',
    'SLOVENŠČINA, BOSANŠČINA': 'Slovenščina',
    'SLOVENŠČINA': 'Slovenščina',
    'Slovenščina/Slovenski': 'Slovenščina',
    'SLOVENŠČINA': 'Slovenščina',
    'SLOVENŠĆINA': 'Slovenščina',
    'SLO': 'Slovenščina',
    'SLOVENŠČINA': 'Slovenščina',
    'SLJ': 'Slovenščina',
    'Bosanščina/Slovenščina': 'Slovenščina',
    'Srbščina/Slovenščina': 'Slovenščina',
    'RUŠČINA/SLOVENŠČINA': 'Slovenščina',
    'SLOVENŠČINA/ITALIJANŠČINA': 'Slovenščina',
    'SLO/HR': 'Slovenščina',
    'Prekmurščina': 'Slovenščina',
    'ITA/SLO': 'Slovenščina',
    'Slovenščina in srbščina': 'Slovenščina',
    'Slovenšina': 'Slovenščina',
    #
    'SRBŠČINA/MAKEDONŠČINA': 'Srb-hr',
    'SEBŠČINA': 'Srb-hr',
    'Srbščina': 'Srb-hr',
    'Srb-hr': 'Srb-hr',
    'SRBŠČINA': 'Srb-hr',
    'HRVAŠČINA': 'Srb-hr',
    'Srbo-hrvaški': 'Srb-hr',
    'Srbski': 'Srb-hr',
    'Hrvaški': 'Srb-hr',
    #
    'RUŠČINA': 'Ukr-Rus',
    'UKRAJINSKI': 'Ukr-Rus',
    'UKRAJINSKI, RUŠČINA': 'Ukr-Rus',
    'RUŠČINA': 'Ukr-Rus',
    'Ruski': 'Ukr-Rus',
    'Ukrajinski': 'Ukr-Rus',
    #
    'MAKEDONŠČINA': 'Makedonščina',
    'MKD': 'Makedonščina',
    'Makedonski': 'Makedonščina',
    'RUŠČINA': 'Makedonščina',
    'RUŠČINA': 'Makedonščina',
    #
    'ALBANŠČINA': 'Albanščina',
    'Albanščina': 'Albanščina',
    'Albanski': 'Albanščina',
    'Shqip': 'Albanščina',
    'Albansko': 'Albanščina',
    'Abanski': 'Albanščina',
    #
    'Bosanski': 'Bosanščina',
    'BOSANSKI': 'Bosanščina',
    'BOSANŠČINA': 'Bosanščina',
    'Bosanski jezik': 'Bosanščina',
    'Bosanski (našinski)': 'Bosanščina',
    #
    'Romski': 'Romski',
    'ROM': 'Romski',
    'ROMSKI': 'Romski',
    'Romščina': 'Romski',
}

# Replace values in 'jezik' column
df['jezik'] = df['jezik'].replace(replace_dict)

df = df[(df['jezik'] != "?") & (df['jezik'] != "")]

Izobrazba matere ipd.

In [54]:
df.dropna(subset=['izob_matere'], inplace=True)
df.dropna(subset=['izob_očeta'], inplace=True)
df.dropna(subset=['število'], inplace=True)
df.dropna(subset=['izpolnjeval'], inplace=True)
df.dropna(subset=['spol'], inplace=True)
df.dropna(subset=['spol_starša'], inplace=True)
df.dropna(subset=['status_matere'], inplace=True)
df.dropna(subset=['status_očeta'], inplace=True)

In [55]:
column_names = df.columns.tolist()

# Print column names
print(column_names)

unique_values = df['izpolnjeval'].unique()

print("Unique values in 'Column1':", unique_values)

['šifra', 'spol', 'ustanova', 'ocenjevalec', 'psiholog', 'datum', 't1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12', 't13', 't14', 't15', 't16', 't17', 't18', 't19', 't20', 't21', 't22', 't23', 't24', 't25', 't26', 't27', 't28', 't29', 't30', 't31', 't32', 't33', 't34', 't35', 't36', 't37', 't38', 't39', 't40', 't41', 't42', 't43', 't44', 't45', 't46', 't47', 't48', 't49', 't50', 't51', 't52', 't53', 't54', 't55', 't56', 't57', 't58', 't59', 't60', 't61', 't62', 't63', 't64', 't65', 't66', 't67', 't68', 't69', 't70', 't71', 't72', 't73', 't74', 't75', 't76', 't77', 't78', 't79', 't80', 'spol_starša', 'izob_matere', 'izob_očeta', 'število', 'izpolnjeval', 'vključen', 'status_matere', 'status_očeta', 'jezik', 'OPP', 'drugo', 'P1', 'opažanja_P1', 'P2', 'opažanja_P2', 'P3', 'opažanja_P3', 'P4', 'opažanja_P4', 'P5', 'opažanja_P5', 'P6', 'opažanja_P6', 'R', 'M', 'P', 'G', 'F', 'A', 'Z', 'S', 'T', 'N', 'B', 'I', 'O', 'K', 'C', 'U', 'L', 'V', 'J', 'Ž', 'D', 'Š', 'E', 'H

In [56]:
# Specified column names to save
columns_to_save = ["spol",'spol_starša', 'izob_matere', 'izob_očeta', 'število', 'izpolnjeval', 'vključen', 'status_matere', 'status_očeta', 'jezik', 'OPP', 'drugo','age_in_months', 'starost_vkljucitev', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19']

# Saving DataFrame to CSV with specified columns
df[columns_to_save].to_csv('precisceno_Ad1.csv', index=False)